# Mount Google Drive

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
from scipy.fftpack import fft
from scipy import signal
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import librosa.display
import librosa.core

import numpy as np
import pandas as pd
import librosa

import torch
from PIL import Image
import torchvision.transforms as transforms # not to be confused with tensorflow
from torchvision.transforms.functional import crop
from io import BytesIO

Mounted at /content/drive


# Create Spectrogram Images

In [ ]:
root = "/content/drive/My Drive/Machine Learning/COSMOS/Final Project/Emotion Recognition/RAVDESS/"

count = 0

for i in range(1, 25):
  actor = "Actor_" + ("0" if i < 10 else "") + str(i) + "/"
  directory = os.fsencode(root + actor)
  for file in os.listdir(directory):
    filename = os.fsdecode(file)
    b = "/content/drive/My Drive/Machine Learning/COSMOS/Final Project/Emotion Recognition/ravimg/"
    y, sr = librosa.load(root + actor + filename)
    y = y[10000:80000] # shorten audio a bit for speed

    window_size = 1024
    window = np.hanning(window_size)
    stft  = librosa.core.spectrum.stft(y, n_fft=window_size, hop_length=512, window=window)
    out = 2 * np.abs(stft) / np.sum(window)

    # For plotting headlessly
    from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

    fig = plt.Figure()
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    p = librosa.display.specshow(librosa.amplitude_to_db(out, ref=np.max), ax=ax, y_axis='log', x_axis='time')
    emotion = filename[6:8]
    fig.savefig(b + emotion + "_" + str(count) + ".jpg")
    count += 1

# Create Dataset

In [6]:
import torch
from PIL import Image
import torchvision.transforms as transforms # not to be confused with tensorflow
from torchvision.transforms.functional import crop
from io import BytesIO

def crop(image):
  return image.crop((80, 58, 577, 428))

transform = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.Lambda(crop),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Optional Sanity Check (comment out transforms.ToTensor())

In [4]:
import matplotlib.image as mpimg

direct = "/content/drive/My Drive/Machine Learning/COSMOS/Final Project/Emotion Recognition/ravimg/"
cntt = 0
for file in os.listdir(direct):
  img = mpimg.imread(direct + file)
  plt.imshow(img)
  plt.show()
  with open(direct + file, 'rb') as f:
    img = f.read()
    image = Image.open(BytesIO(img))
    image = transform(image)
  plt.imshow(image)
  plt.axis('off')
  plt.show()
  cntt += 1
  #print(cntt)
  if cntt == 5:
    break

Output hidden; open in https://colab.research.google.com to view.

Save to .pt file

In [9]:
root_path = "/content/drive/My Drive/Machine Learning/COSMOS/Final Project/Emotion Recognition/"
folder = "ravimg/"

directory = os.fsencode(root_path + folder)
count = 0

features = []
labels = []
cnt = 0

for file in os.listdir(directory):
  filename = os.fsdecode(file)
  with open(root_path + folder + filename, 'rb') as f:
    img = f.read()
    image = Image.open(BytesIO(img))
    image = transform(image)
    features.append(image)
  emote = filename[0:2]
  emote = int(emote) - 1
  labels.append(torch.tensor([emote]))

features = torch.stack(features)
labels = torch.stack(labels)
dset = torch.utils.data.TensorDataset(features, labels)
torch.save(dset, "dset.pt")